In my previous [notebook](https://www.kaggle.com/code/mbmmurad/mp3-to-wav-conversion), we tried to convert mp3 audio files to wav files. But it took almost 25 minutes to convert the Validation files. In this notebook we'll try a different way to convert the mp3 files to wav files using joblib. This notebook takes only 8-9 minutes to process the validation files. 
(3times faster)



**Reference** :

Idea is from this notebook : https://www.kaggle.com/code/kingabzpro/asr-mp3-to-wav-dataset?fbclid=IwAR3D61p3PW9lY-IGTOLrjf52b1_SQkc2FFuHl9kEbGumVlgHLqVOGJoQl5w

# Importing Necessary Libraries

In [1]:
import os
import cv2
import skimage.io
from tqdm.notebook import tqdm
import zipfile
import pandas as pd
import numpy as np
import shutil

from pydub import AudioSegment
from joblib import Parallel, delayed

In [2]:
ROOT_PATH = "../input/dlsprint/train_files"
OUTPUT_DIR = "./train_files_wav"

In [3]:
os.mkdir(OUTPUT_DIR)

In [4]:
def save_fn(filename):
    
    path = f"{ROOT_PATH}/{filename}"
    save_path = f"{OUTPUT_DIR}"
    if not os.path.exists(save_path):
        os.makedirs(save_path, exist_ok=True)
    
    if os.path.exists(path):
        try:
            sound = AudioSegment.from_mp3(path)
            sound = sound.set_frame_rate(16000)
            sound.export(f"{save_path}/{filename[:-4]}.wav", format="wav")
        except:
            print(path)

In [5]:
#-------------------------------
# imports
#-------------------------------
import os 
import pandas as pd 
from tqdm.auto import tqdm
import warnings
import librosa
import io
import soundfile as sf
tqdm.pandas()
warnings.filterwarnings('ignore')


In [6]:
#---------------
# data filtering
#---------------
def filter_votes(x):
    up=x["up_votes"]
    down=x["down_votes"]
    if up-down<=0:
        return None
    elif up==0:
        return None
    else:
        return up
# ------------------------- train data----------------------------------------
train_df=pd.read_csv("../input/dlsprint/train.csv")
print("Total Data before filtering:",len(train_df))
train_df["up_votes"]=train_df.progress_apply(lambda x:filter_votes(x),axis=1)
train_df.dropna(subset = ['up_votes'],inplace=True)
print("Total Data after filtering:",len(train_df))
audio_files=train_df["path"].tolist()
len(audio_files)


Total Data before filtering: 206950


  0%|          | 0/206950 [00:00<?, ?it/s]

Total Data after filtering: 37404


37404

In [7]:
import time
start = time.time()

Parallel(n_jobs=8, backend="multiprocessing")(
    delayed(save_fn)(filename) for filename in tqdm(audio_files)
)

end = time.time()
print("total time to process: {x} seconds".format(x=end-start))

  0%|          | 0/37404 [00:00<?, ?it/s]

total time to process: 2602.5732564926147 seconds
